## Simulación de MonteCarlo

In [17]:
import numpy as np
from numpy.linalg import cholesky
from MC_cerrada import *

np.random.seed(77)

In [18]:
S_0 = np.array([1,1,1])
num_steps = 20
vols = np.array([0.29,0.38,0.33]).reshape(-1,1)
risk_free_rate = 0.01


TTM = initial_maturity = 5 #years
value_date_index = 5 #actual step
num_asian_dates = num_steps + 1 #Time steps + 1
init_time_array = np.linspace(0, initial_maturity, num_asian_dates)

rho = 0.1
correl_matrix = rho = np.array([
    [1, 0.275125, 0.329366],
    [0.275125, 1, 0.381295],
    [0.329366, 0.381295, 1]
]) 


num_sims = 100000

num_assets = 3 #numero de activos

rets, payoff, premium = MonteCarlo(initial_maturity, S_0, num_sims, num_assets, num_asian_dates, value_date_index, correl_matrix,risk_free_rate, vols)

In [19]:
print(rets)

[[[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]
  [1.02057297 0.53319234 0.88871561]
  ...
  [0.63693688 0.17206475 0.26833344]
  [0.67225676 0.20161834 0.33230861]
  [0.65371429 0.1524178  0.36082416]]

 [[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]
  [1.02057297 0.53319234 0.88871561]
  ...
  [1.72532464 0.77897512 1.8417265 ]
  [1.65849154 0.74640983 2.01065259]
  [1.30205716 0.53851687 1.6793864 ]]

 [[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]
  [1.02057297 0.53319234 0.88871561]
  ...
  [0.99908574 0.34057256 0.41510893]
  [0.84424215 0.2998968  0.37949184]
  [0.87228096 0.39809851 0.42524684]]

 ...

 [[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]
  [1.02057297 0.53319234 0.88871561]
  ...
  [1.45286713 2.8783833  1.58558192]
  [1.51350801 3.63023775 1.70230657]
  [1.18487081 3.09406966 1.64915344]]

 [[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]


In [20]:
rets.shape

(100000, 20, 3)

In [21]:
np.savez_compressed('Datos/Transformados/montecarlo_data.npz', features=rets, target=payoff)

### Valor presente

Dados todos los timesteps:

In [22]:
#present_value (numpy.ndarray): present value of the payoff in every time step. Shape (num_sims, num_asian_dates-1)

time_array = np.linspace(0, initial_maturity, num_steps)

present_value = np.matmul(payoff.reshape(-1,1),np.exp(- risk_free_rate*(initial_maturity-time_array )).reshape(1,-1))

In [23]:
print(present_value)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.05920886 0.05936488 0.05952131 ... 0.06191782 0.06208098 0.06224456]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.33679166 0.33767912 0.33856892 ... 0.35220076 0.35312883 0.35405934]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
